# Example of FaIRv2.1

FaIR v2.1 is object-oriented and designed to be more flexible than its predecessors. This does mean that setting up a problem is different to before - gone are the days of 60 keyword arguments to `fair_scm` and we now use hierarchical classes with fewer arguments that in the long run should be easier to use. Of course, there is a learning curve, and will take some getting used to. This tutorial aims to walk through a simple problem using FaIR 2.1.

The structure of FaIR 2.1 centres around the `FAIR` class, which contains all information about the scenario(s), the forcer(s) we want to investigate, and any configurations specific to each species and the response of the climate.

A run is defined as follows:

```
fair = FAIR(scenarios, configs, time, run_config)
fair.run()

# results are stored within fair, and can be obtained such as
print(fair.temperature)
```

Multiple `scenarios` and `configs` can be supplied in each run of FaIR, and due to internal parallelisation is the fastest way to run the model. The total number of scenarios that will be run is the product of `scenarios` and `configs`. For example we might want to run three emissions `scenarios` -- let's say SSP1-2.6, SSP2-4.5 and SSP3-7.0 -- using climate calibrations (`configs`) from the UKESM, GFDL, MIROC and NorESM climate models. This would give us a total of 3$\times$4 = 12 ensemble members in total which are run in parallel.

The most difficult part to learning FaIR 2.1 is correctly defining the `scenarios` and `configs`, and allows for a lot of flexibility. In this tutorial the recommended order in which to define a problem is set out.

## Defining the scope of our problem

### The SpeciesID

The starting point is the `SpeciesID` class which defines the forcers -- anthropogenic or natural -- that are present in your scenario. Each `SpeciesID` is assigned a name that is used to distinguish it from other species, along with a `Category` class that defines its general behaviour, and a `RunMode` that defines how we want to include this particular `SpeciesID` in the model.

In [ ]:
from fair21 import SpeciesID, Category, RunMode

In [ ]:
help(SpeciesID)

In this example we'll start off running a scenario with CO2 from fossil fuels and industry, CO2 from AFOLU, CH4, N2O, and Sulfur. To highlight some of the functionality we'll run CO2 and Sulfur emissions-driven, and CH4 and N2O concentration-driven. (This is akin to an `esm-ssp585` kind of run from CMIP6, though with fewer species).

First, we need to define the `SpeciesID`s of all of the things we want to include. Full simulations may have 50 or more species included, so it can be beneficial to build a `dict` of `SpeciesID`s, particularly as we will re-use them later. This is what we'll do here.

In [ ]:
species_ids = {}
species_ids['co2_ffi'] = SpeciesID('CO2 Fossil Fuel and Industry', Category.CO2_FFI, RunMode.EMISSIONS)
species_ids['co2_afolu'] = SpeciesID('CO2 AFOLU', Category.CO2_AFOLU, RunMode.EMISSIONS)
species_ids['co2'] = SpeciesID('CO2', Category.CO2, RunMode.FROM_OTHER_SPECIES)
species_ids['ch4'] = SpeciesID('CH4', Category.CH4, RunMode.CONCENTRATION)
species_ids['n2o'] = SpeciesID('N2O', Category.N2O, RunMode.CONCENTRATION)
species_ids['sulfur'] = SpeciesID('Sulfur', Category.SULFUR, RunMode.EMISSIONS)

There's a few things going on here. We see that `Category` and `RunMode` have specific values that they can take: they are `dataclass`es (a new feature in Python 3.7, so FaIR 2.1 requires at least this version of Python). Let's explore these a litte more closely.

### Category class

We see there's 20 or valid values for `Category` which can be attached to a `SpeciesID` or `Species`, and everything included in FaIR must fall into one of these categories:

In [ ]:
dir(Category)

Trying to access a `Category` outside of this raises an error. But miscellaneous forcings that are not part of the above categorization can be included as`Category.OTHER` (cf. FaIR v1.0 considering non-CO2 forcings).

In [ ]:
try:
    Category.COSMIC_RAYS
except Exception as exc:
    print(f"{str(exc)} is not a valid Category")

### RunMode class

The `RunMode` defines how we want to drive the model for each `Species`. It is defined as a attribute of `SpeciesID` and takes one of four possible values:

1. `RunMode.EMISSIONS`
2. `RunMode.CONCENTRATION`
3. `RunMode.FORCING`
4. `RunMode.FROM_OTHER_SPECIES`

The first three are self-explanatory. `RunMode.FROM_OTHER_SPECIES` is used when there's no direct emission or concentration relationship to a climate forcing agent.

In our example above, we use `RunMode.FROM_OTHER_SPECIES` for CO2 because the total burden of atmospheric CO2 comes from two sources: fossil fuel + industry and AFOLU. In reality this is true for many emissions sources, but CO2 AFOLU has a special treatment in FaIR and as such when running emissions-driven it is necessary to separate the sources.

Not all `RunMode`s are defined or sensible for all `Species`.

In [ ]:
try:
    SpeciesID('Solar', Category.SOLAR, RunMode.EMISSIONS)
except Exception as exc:
    print(str(exc))

and in fact, we can't run just the plain `Category.CO2` mode emissions-driven either.

In [ ]:
try:
    SpeciesID('Total CO2', Category.CO2, RunMode.EMISSIONS)
except Exception as exc:
    print(str(exc))

That's not to say that `Category.CO2` can't take emissions: when we provide a concentration-driven CO2 run, we have no idea what the source of the CO2 actually was so we have to assign it all to one group. We'll revisit this when we come to concentration-driven CO2 runs.

## Assigning data: the Species class

`SpeciesID` just tells us what is present. To build a `Scenario`, we have to populate it with `Species` that assigns emissions, concentrations or forcing to a `SpeciesID`. We can run multiple `Scenario`s per evaluation of FaIR, and therefore we can have several instances of `Species` pointing to the same `SpeciesID` when running a scenario batch. This is what we will do below, running two scenarios.

In [ ]:
from fair21 import Species
import numpy as np

In [ ]:
help(Species)

Here, we'll run two 50 year long scenarios. The first will have emissions and concentrations for the 50 years that roughly reflect present-day emissions or concentrations to each `Species`. The second will ramp up to that level in year 50 from pre-industrial levels.

Again, we'll use `dict`s to group together `Species`, this time nesting them by scenario.

Note also that we need to define the total CO2 `Species`, but we do not assign emissions to it.

For CH4 and N2O (and in fact, every greenhouse gas), the pre-industrial concentrations are stored as `baseline_concentrations` in a module of species defaults, and can be imported.

In [ ]:
from fair21.defaults.data.default_species_config import default_species_config

ch4_concentration_pi = default_species_config['ch4'].baseline_concentration
n2o_concentration_pi = default_species_config['n2o'].baseline_concentration
print(ch4_concentration_pi, n2o_concentration_pi)

# for a list of defined defaults
print(default_species_config.keys())

In [ ]:
species = {}

species['abrupt'] = {}
species['ramp'] = {}

species['abrupt']['co2_ffi'] = Species(species_ids['co2_ffi'], emissions=np.ones(50)*38) # default unit GtCO2/yr
species['abrupt']['co2_afolu'] = Species(species_ids['co2_afolu'], emissions=np.ones(50)*3) 
species['abrupt']['ch4'] = Species(species_ids['ch4'], concentration=np.ones(50)*1800) # default unit ppb
species['abrupt']['n2o'] = Species(species_ids['n2o'], concentration=np.ones(50)*325)
species['abrupt']['sulfur'] = Species(species_ids['sulfur'], emissions=np.ones(50)*100) # default unit MtSO2/yr
species['abrupt']['co2'] = Species(species_ids['co2'])

species['ramp']['co2_ffi'] = Species(species_ids['co2_ffi'], emissions=np.linspace(0, 38, 50)) # default unit GtCO2/yr
species['ramp']['co2_afolu'] = Species(species_ids['co2_afolu'], emissions=np.linspace(0, 3, 50)) 
species['ramp']['ch4'] = Species(species_ids['ch4'], concentration=np.linspace(ch4_concentration_pi, 1800, 50))
species['ramp']['n2o'] = Species(species_ids['n2o'], concentration=np.linspace(n2o_concentration_pi, 325, 50))
species['ramp']['sulfur'] = Species(species_ids['sulfur'], emissions=np.ones(50)*100) # default unit MtSO2/yr
species['ramp']['co2'] = Species(species_ids['co2'])

We see that `SpeciesID`s are re-used as they uniquely define what each substance actually is, but `Species` are defined once per `Scenario` as we can investigate different forcing pathways with the same setup.

Let's see what the `Species` class looks like to the code.

In [ ]:
species['ramp']['n2o']

## Defining Scenarios

We have declared all of our `Species`, and we now need to formally assign them to `Scenario`s in the code.

In [ ]:
from fair21 import Scenario

In [ ]:
help(Scenario)

Again we'll declare scenarios as a dict, and can take a peek at what our `Scenario` actually looks like to FaIR.

In [ ]:
scenarios = {}
for scenario in ['abrupt', 'ramp']:
    scenarios[scenario] = Scenario(scenario, [specie for specie in species[scenario].values()])

In [ ]:
scenarios['ramp']

## The Configs

`Scenario`s define what goes into the model, but `Config`s define the model behaviour. There are two types of configs (well, actually three if you include the overall run configs, but these are defined at the top level and we'll revisit these later):

1. `ClimateResponse` defines the climate sensitivity, ocean heat capacities, heat transfer coefficient, etc. It defines how forcing is converted to temperature in the model. Only one `ClimateResponse` per config should be defined.
2. `SpeciesConfig` defines things such as greenhouse gas lifetimes, radiative forcing efficacy, gas cycle airborne fraction, and many other things. It has a lot of options, and a lot of flexibility. A list containing a `SpeciesConfig` for each defined species in the scenarios needs to be provided.

`Config`s are defined as follows:

```
config = Config(name, climate_response, species_configs)
```

We then provide a list of configs to FaIR.

In this example, we'll provide three configs representing high, medium and low climate sensitivity climate models, and also vary the `SpeciesConfig` in each to provide some interesting feedbacks.

### Climate response

We'll start with the climate response which is slightly easier. This is an $n$-box energy balance model, where $n=3$ dy default (this can be changed in the `RunConfig`). 

In [ ]:
from fair21 import ClimateResponse

In [ ]:
print(ClimateResponse.__doc__)